In [ ]:
%pip install -qU  langchain_milvus
%pip install -qU langchain-openai

In [8]:
import os
import json

from typing import Optional, List, Any

from pymilvus import connections, utility

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_milvus import Milvus


class MilvusStorage:
    def __init__(self, collection_name):
        self.milvus_url = os.getenv("MILVUS_URL")
        self.embeddings_model = OpenAIEmbeddings(model=os.getenv("EMBEDDINGS_MODEL"), api_key=os.getenv("OPENAI_API_KEY"))
        self.collection_name = collection_name
    
    def use_collection(self, partition_key_field: str = "namespace") -> Optional[Milvus]:
        try:
            connections.connect("default", uri=self.milvus_url) 
            collections = utility.list_collections()
            if self.collection_name in collections:
                drop_old = False
            else:
                drop_old = True
            connections.disconnect("default")

            return Milvus(
                self.embeddings_model,
                collection_name=self.collection_name,
                connection_args={"uri": self.milvus_url},
                drop_old=drop_old,
                auto_id=False,
                primary_field="primary_key", 
                index_params={"metric_type": "COSINE"}, 
                partition_key_field=partition_key_field # Use the "namespace" field as the partition key
            )
            # return vectorstore
        except Exception as e:
            print(f"Error for use a collection: {e}")
            return None

    def upsert_files(self, vectorstore: Milvus, docs: List[Document]) -> Optional[List[str]]:
        if docs is None or len(docs) == 0:
            print("No documents to upsert.")
            return None
        try:
            uuids = [str(hash(json.dumps(doc.metadata))) for doc in docs]
            vectorstore.delete(uuids)
            ids = vectorstore.add_documents(docs, ids=uuids)
            return ids
        except Exception as e:
            print(f"Error upserting documents: {e}")
        

    def create_retriver(self, vectorstore: Milvus, namespace: str) -> Any:
        try:
            retriver = vectorstore.as_retriever(
                search_kwargs = {
                    "expr": f"namespace == '{namespace}'",
                    "k": 2, 
                    'metric_type': "COSINE",
                    'score_threshold': 0.8
                }
            )
            return retriver
        except Exception as e:
            print(f"Error creating RAG: {e}")

In [9]:
docs = [
    Document(metadata={'namespace': 'jorgeang33@gmail.com', 'source': 'example.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez'}, page_content='Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de cualquier marcador de posición (como este, por\nejemplo) y comience a escribir.\nPara aplicar facilmente cualquier formato de texto que vea en esta pagina, vaya al grupo\nEstilos, que encontrara en la pestana Inicio de la cinta de opciones.\n¢ Desea insertar una imagen de sus archivos o agregar una forma o un cuadro de texto?\n¡Adelante! En la pestaña Insertar de la cinta de opciones, pulse la opción que necesite.\n'),
    Document(metadata={'namespace': 'jorgeang33@gmail.com', 'source': 'example.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez'}, page_content='Esto es texto que esta en una imagen, en la segunda\npagina del documento, a continuación, información sobre\nlos perros:\nEl perro, llamado perro doméstico o can, y en algunos\nlugares coloquialmente llamado chucho, tuso, choco,\nentre otros; es un mamifero carnivoro de la familia de los\ncanidos, que constituye una especie del género Canis.\n')
]

milvus = MilvusStorage(collection_name="example")
vector_storage = milvus.use_collection()
milvus.upsert_files(vector_storage, docs)

['-9135271458773154252', '-7961177023158993311']

In [11]:
docs = [
    Document(metadata={'namespace': 'jorgeang33@gmail.com', 'source': 'example2.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez'}, page_content='Texto de la primera pagina del documento\n'),
    Document(metadata={'namespace': 'jorgeang33@gmail.com', 'source': 'example2.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez'}, page_content='Otro texto de la segunda pagina del documento\n')
]
milvus.upsert_files(vector_storage, docs)

['-8476804555162894089', '-3201606266193177534']

In [22]:
retriver = storage.create_retriver(vectorstore, 'jorgeang33@gmail.com')
retriver.invoke("texto de la segunda pagina")

[Document(metadata={'pk': '-9088355280838447519', 'namespace': 'jorgeang33@gmail.com', 'source': 'example.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez'}, page_content='Esto es texto que esta en una imagen, en la segunda\npagina del documento, a continuación, información sobre\nlos perros:\nEl perro, llamado perro doméstico o can, y en algunos\nlugares coloquialmente llamado chucho, tuso, choco,\nentre otros; es un mamifero carnivoro de la familia de los\ncanidos, que constituye una especie del género Canis.\n'),
 Document(metadata={'pk': '9173623458401502104', 'namespace': 'jorgeang33@gmail.com', 'source': 'example.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez'}, page_content='Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de cualquier marcador de posición (como este, por\nejemplo) y comience a escribir.\nPara aplicar facilmente cualquier formato de texto que vea en esta pagina, vaya al grupo\nEstilos, que encontrara en la pestana Inicio de la cinta d

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="milvus_retriever",
    description="A retriever tool using Milvus",
)

tools = [retriever_tool]